## Preprocess and save training data

In [3]:
import sys
sys.path.append("../../src")

from tree_parser import file_contents, meta_math_database

# from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
# from my_utils import get_proof_steps_graph, print_proof_steps_graph

import json


from expanding import construct_proof, expand_all_nodes_with_depth

# from collections import defaultdict

# import glob

# import os

# import numpy as np

# from sklearn.metrics import roc_auc_score

# import random

from tqdm import tqdm

import time

# from expanding import PStep, construct_proof
# from theorem_database import TheoremDatabase

from sklearn.model_selection import train_test_split

import os

In [4]:
%%time

text = file_contents("../../data/set_mod.mm")
database = meta_math_database(text,n=7000)

included 3651825 tokens from ../../data/set_mod.mm
proposition: 7000CPU times: user 1min 34s, sys: 19.7 s, total: 1min 53s
Wall time: 2min 10s


In [5]:
test_props = [
    "dfss3",
    "dfss2",
    "dfss",
    "t1lucas",
    "t2lucas",
    "ssun1",
    "t4lucas",
    "t5lucas",
    "opoe",
    "omoe",
    "pwin",
    "inidm",
    "in0",
    "sstr",
    "ssequn1",
    "pwunss",
    "epee",
    "emee",
    "oddp1eveni",
    "dvdsadd2b",
    "opoeALTV",
    "omoeALTV"
]

#Check if something is absent
for t in test_props:
    if t not in database.propositions.keys():
        print(f"{t} not in database.")

opoe not in database.
omoe not in database.
epee not in database.
emee not in database.
oddp1eveni not in database.
dvdsadd2b not in database.
opoeALTV not in database.
omoeALTV not in database.


In [6]:
train_props = [p for p in database.propositions.keys() if p not in test_props]
print(len(train_props))

6986


In [7]:
train_props, valid_props = train_test_split(train_props, test_size=0.2, random_state=2)
print(len(train_props), len(valid_props))

5588 1398


In [8]:
def expand_all_nodes_with_depth_and_filter(root, target_depth, filter_list):
    if root.depth == target_depth and root.label in filter_list:
        root = root.expand()
    
    for i in root.inputs:
        expand_all_nodes_with_depth_and_filter(i, target_depth, filter_list)
        
    return root

In [9]:
# prop_label = "dfss2"#train_props[987]
# expand_filter = train_props

# prop_dataset = {
#     'id':prop.number,
#     'prop': prop_label,
#     'steps':[],
#     'links':[]
# }

# prop = database.propositions[prop_label]

# prop_proof = construct_proof(prop)

# for s in prop_proof.get_steps_df():
#     print(s.label, s.label in expand_filter)



# prop_proof = expand_all_nodes_with_depth_and_filter(prop_proof, 0, expand_filter)


# prop_dataset

# prop_proof.print_graph()

In [10]:
def get_prop_dataset_with_filter(prop_label, filter_list):
    prop = database.propositions[prop_label]
    
    prop_dataset = {
        'id':prop.number,
        'prop': prop_label,
        'steps':[],
        'links':[]
    }
    
    prop_proof = construct_proof(prop)
    prop_proof = expand_all_nodes_with_depth_and_filter(prop_proof, 0, filter_list)

    #Populate step numbers
    next_step_n = 0
    
    for s in prop_proof.get_steps_df(): 
        next_step_n += 1
        s._step_num = next_step_n
    
    #Populate data
    
    _no_zero_labels = True 
    #This flag is necessary to remove from the dataset propositions which every label is true
    #All labels true doesnt contribute much to the train/validation
    
    for s in prop_proof.get_steps_df():
        #Since negative class is some depth bigger than 0, in case we have some, set this to false
        if s.statement_depth > 0: _no_zero_labels = False
        
        prop_dataset["steps"].append((
            s._step_num,
            s.label,
            s.raw_statement,
            s.raw_prop_statement,
            int(s.statement_depth == 0)
        ))
        
        for child_s in s.inputs:
            prop_dataset["links"].append((child_s._step_num, s._step_num))
        
    if _no_zero_labels:
        raise Exception(f"Only positive classes in {prop_label}.")
        
    return prop_dataset

In [11]:
%%time

## Process training data

train_props_data = []
train_failed_props = []

for i, p in enumerate(train_props):
    print(f"\r{i+1}/{len(train_props)}", end="")
    
    if len(database.propositions[p].entails_proof_steps) == 0:
        continue
        
    try:
        train_props_data.append(get_prop_dataset_with_filter(p, train_props))
    except:
        train_failed_props.append(p)
      
print()
print(len(train_props_data))
print(len(train_failed_props))

5588/5588
5108
248
CPU times: user 2min 38s, sys: 42 s, total: 3min 20s
Wall time: 5min 5s


In [12]:
%%time

## Process validation data

valid_props_data = []
valid_failed_props = []

for i, p in enumerate(valid_props):
    print(f"\r{i+1}/{len(valid_props)}", end="")
    
    if len(database.propositions[p].entails_proof_steps) == 0:
        continue
        
    try:
        valid_props_data.append(get_prop_dataset_with_filter(p, valid_props))
    except:
        valid_failed_props.append(p)
      
print()
print(len(valid_props_data))
print(len(valid_failed_props))

1398/1398
817
530
CPU times: user 3.7 s, sys: 352 ms, total: 4.05 s
Wall time: 4.78 s


In [13]:
ts = time.time()
os.mkdir(f"../../data/datasets/{ts}")

json.dump(train_props_data, open(f"../../data/datasets/{ts}/train.json", "w"))
json.dump(valid_props_data, open(f"../../data/datasets/{ts}/valid.json", "w"))

In [14]:
open(f"../../data/datasets/{ts}/test.txt", "w").write("\n".join(test_props))

148